In [85]:
import bs4
from collections import defaultdict
from pprint import pprint

In [152]:
with open('./ItemPages/html/49544.html','r') as f:
    html = f.read()
soup = bs4.BeautifulSoup(html,'lxml')

# Breaks screw up some parsing, and don't think I need them, so just replace
for br in soup('br'):
    br.replace_with('\n')
    
# Italics screw up parsing of references, so getting rid of them for now
for ii in soup('i'):
    ii.replace_with(ii.get_text())


### Remove some tags so it's easier to see the rest

In [153]:
[x.extract() for x in soup.findAll('script')]
[x.extract() for x in soup.findAll('noscript')]
[x.extract() for x in soup.findAll('iframe')]
[x.extract() for x in soup.findAll('svg')]
[x.extract() for x in soup.findAll('style')]
[x.extract() for x in soup.select('#nas-footer')]
[x.extract() for x in soup.select('.nas')]
[x.extract() for x in soup.select('.artwork__interaction')];


In [154]:
print(soup.select('body')[0].prettify())

<body>
 <!-- Google Tag Manager (noscript) -->
 <!-- End Google Tag Manager (noscript) -->
 <!-- Google Tag Manager -->
 <link href="/dist/met-nas-generated.css?v=202005070953" rel="stylesheet"/>
 <div class="content-wrapper content-wrapper--search-nav">
  <div class="artwork__search-options-nav">
   <a class="banner__link-back" href="/art/collection">
    Browse the Collection
   </a>
  </div>
  <div class="artwork__search-results-nav">
  </div>
 </div>
 <section class="animal-crossing-modal ac-modal js-ac-modal">
  <a class="js-ac-modal__close ac-modal__close" href="#">
   ×
  </a>
  <h3>
   Crop your artwork:
  </h3>
  <canvas class="ac-modal__canvas" id="imageToEncode">
  </canvas>
  <div class="ac-modal__image-container">
   <img alt="Image To Crop" class="js-ac-image ac-modal__image" src=""/>
  </div>
  <h3>
   Scan your QR code:
  </h3>
  <img alt="QR Code for Animal Crossing New Horizons" class="ac-modal__qr" id="qrCode"/>
  <p class="ac-modal__credit">
   Gratefully built with

### Set up dictionary to hold data for this page

Using `defaultdict` so can default to a string and append any data that we run across under the same field name, or on separate lines in the accordions

In [155]:
data = defaultdict(str)

#### Title

In [156]:
current_key = 'Title'
data[current_key] += soup.select_one('.artwork__title--text').get_text().strip()
print(data[current_key])

Seated Falconer


#### Date

In [157]:
current_key = 'Date'
data[current_key] += soup.select_one('.artwork__date time').get_text().strip()
print(data[current_key])

mid-to-late 6th century


#### Name (can be artist or designer...)

If there's a modifier it's in text with newline in between?

In [158]:
current_key = 'Name'
data[current_key] += soup.select_one('.artwork__artist__name').get_text().strip()
print(data[current_key])

China


#### Artist region

In [159]:
current_key = 'Artist_region'
data[current_key] += soup.select_one('.artwork__artist__region').get_text().strip()
print(data[current_key])

#### Description

In [160]:
current_key = 'Description'
data[current_key] += soup.select_one('.artwork__intro__desc').get_text().strip()
print(data[current_key])

#### Location

In [161]:
current_key = 'Location'
if soup.select_one('.artwork__location--message'):
    data[current_key] += soup.select_one('.artwork__location--message').get_text().strip()
    data[current_key] += "|"
    data[current_key] += soup.select_one('.artwork__location--gallery').get_text().strip()
else:
    data[current_key] += soup.select_one('.artwork__location').get_text().strip()
print(data[current_key])

On view at The Met Fifth Avenue in|Gallery 207


#### Object details

In [163]:
for ss in soup.select('.artwork-info .artwork__tombstone--row'):
    label = 'unlabeled'
    value = ''
    if ss.select_one('.artwork__tombstone--label'):
        label = ss.select_one('.artwork__tombstone--label').get_text().rstrip(':')
    if ss.select_one('.artwork__tombstone--value'):
        value = ss.select_one('.artwork__tombstone--value').get_text().strip().replace('\n','|')
    current_key = label
    if data[current_key]: data[current_key] += "|"
    data[current_key] += value
    print(current_key, ':', data[current_key])

misc : 北魏/北齊 彩繪陶馴鷹人俑
Title : Seated Falconer|Seated Falconer
Period : Northern Wei (386–534)–Northern Qi (550–577) dynasty
Date : mid-to-late 6th century|mid-to-late 6th century
Culture : China
Medium : Earthenware with red and white pigments
Dimensions : H. 6 1/8 in. (15.6 cm); W.  6 in. (15.2 cm)
Classification : Tomb Pottery
Credit Line : Gift of Florence and Herbert Irving, 2015
Accession Number : 2015.500.7.6


#### Component accordions

In [164]:
current_key = 'none'
def content_breakout(content,current_key):
    for cc in content.children:
        if isinstance(cc, bs4.element.Tag):
            # print('(tag)', cc)
            if cc.name == 'strong':
                current_key = cc.get_text().strip(':')
                print('(strong)', cc.get_text())
            elif cc.name == 'br':
                # print('break')
                continue
            elif cc.name == 'a':
                if data[current_key]: data[current_key] += "|"
                data[current_key] += cc['href'] + '–' + cc.get_text()
                print(cc['href'], '|', cc.get_text())
            elif cc.has_attr('class') and any([xx.endswith('list') for xx in cc['class']]):
                print('LIST')
                content_breakout(cc.p,current_key)
        elif isinstance(cc, bs4.element.NavigableString):
            if len(cc.strip())>0:
                if data[current_key]: data[current_key] += "|"
                data[current_key] += cc.strip()
                print('(--)', cc.strip())
        else:
            print('**other!**', cc)
    print('---------------\n')

for ss in soup.select('.component__accordions .accordion'):
    title = ss.select_one('.accordion__title').get_text()
    current_key = title
    print('>>', title)
    content = ss.select_one('.accordion__content')
    content_breakout(content,current_key)

>> Provenance
(--) Provenance Information: Purchased from R. H. Ellsworth, Ltd., New York, by Florence and Herbert Irving, New York, in 1988; Collection of Florence and Herbert Irving, New York from 1988 to 2015, on loan to The Metropolitan Museum of Art from 1988 until donation to The Metropolitan Museum of Art in 2015. 

Exhibition Information: On view continuously since 1988 in the Charlotte C. Weber Galleries for the Arts of Ancient China at The Metropolitan Museum of Art.

Publication Information: Published on The Metropolitan Museum of Art’s online collection database.(http://www.metmuseum.org/art/collection) as of July 2015.
(--) [ R. H. Ellsworth Ltd. , New York, by 1988, sold to Irving]; Florence and Herbert Irving , New York (1988–2015, on loan to MMA from 1988 until donation to MMA in 2015)
---------------

>> Exhibition History
(--) New York. The Metropolitan Museum of Art. "Arts of Ancient China," 2005.
---------------

>> Timeline of Art History
LIST
(strong) Timelines
ht

#### Related objects

In [165]:
current_key = 'Related_objects'
for aa in soup.select('a.gtm__relatedartwork:not(:has(> img))'):
    print(aa['href'].split('/')[-1], ':', aa.get_text())
    if data[current_key]: data[current_key] += "|"
    data[current_key] += aa['href'].split('/')[-1] + '–' + aa.get_text()

49545 : Seated Falconer
42400 : Flask
39649 : Bowl with Dragons among Waves

65100 : Standing Attendant
44795 : Camel


### Remove newlines from entries

In [167]:
for k,v in data.items():
    data[k] = v.replace('\n',' ')

In [168]:
pprint(data)

defaultdict(<class 'str'>,
            {'Accession Number': '2015.500.7.6',
             'Artist_region': '',
             'Classification': 'Tomb Pottery',
             'Credit Line': 'Gift of Florence and Herbert Irving, 2015',
             'Culture': 'China',
             'Date': 'mid-to-late 6th century|mid-to-late 6th century',
             'Description': '',
             'Dimensions': 'H. 6 1/8 in. (15.6 cm); W.  6 in. (15.2 cm)',
             'Exhibition History': 'New York. The Metropolitan Museum of Art. '
                                   '"Arts of Ancient China," 2005.',
             'Location': 'On view at The Met Fifth Avenue in|Gallery 207',
             'Medium': 'Earthenware with red and white pigments',
             'Name': 'China',
             'Period': 'Northern Wei (386–534)–Northern Qi (550–577) dynasty',
             'Provenance': 'Provenance Information: Purchased from R. H. '
                           'Ellsworth, Ltd., New York, by Florence and Herbert '
    

### testing a problem area

In [3]:
import bs4
with open('./html/35.html', 'r') as f:
    html = f.read()
soup = bs4.BeautifulSoup(html,'lxml')

`\x0A` and `\x0D` as \r and \n

In [6]:
soup.select_one('.artwork__artist__name').get_text().strip().replace('\x0A',' ')

'Designed by                         Paulding Farnham'